In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import re
import matplotlib.pyplot as plt
from matplotlib import *
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score,learning_curve, GroupKFold
from sklearn.metrics import confusion_matrix



In [ ]:
df =  pd.read_csv('../2017_07_07/TADPOLE_D1_D2.csv',low_memory=False)

In [ ]:
df.head(0)


In [ ]:
print(df['DX'].head())
print(df['DX_bl'].head())
print(df['EXAMDATE_BAIPETNMRC_09_12_16'].unique())

In [ ]:
df_sample = df.sample(frac=0.1, random_state=1)
pd.set_option('display.max_columns', None)
name_columns=df.head(0)
#EXAMDATE	DX_bl	DXCHANGE	AGE	PTGENDER	PTEDUCAT	PTETHCAT	PTRACCAT	PTMARRY	
print(df['PTETHCAT'].head(10))

In [ ]:
print(df['DXCHANGE'].head(1))
'1=Stable:NL to NL, 2=Stable:MCI to MCI, 3=Stable:AD to AD, ' 
'4=Conv:NL to MCI, 5=Conv:MCI to AD, 6=Conv:NL to AD, ' 
'7=Rev:MCI to NL, 8=Rev:AD to MCI, 9=Rev:AD to NL, -1=Not available'

In [ ]:

#data = df[['PTGENDER','AGE','PTEDUCAT','APOE4','DXCHANGE','PTETHCAT','PTMARRY','PTEDUCAT','PTRACCAT','EXAMDATE']].copy()
data = df
print(data.shape)
print(data['DX_bl'].unique())

#data.dropna(axis=0,inplace= True)
data['PTGENDER']= data['PTGENDER'].replace(['Male','Female'],[1,0])
data['APOE4']= data['APOE4'].astype(float)
data['AGE']= data['AGE'].astype(float)
data['PTETHCAT']= data['PTETHCAT'].replace(['Hisp/Latino','Not Hisp/Latino','Unknown'],[2,1,0]).astype(int)
data['PTMARRY']= data['PTMARRY'].replace(['Married','Widowed','Divorced','Never married','Unknown'],[0,1,2,3,4]).astype(int)
data['PTRACCAT'] = data['PTRACCAT'].replace(['White','More than one','Black','Asian', 'Am Indian/Alaskan','Hawaiian/Other PI', 'Unknown'],[0,1,2,3,4,5,6]).astype(int)
data['DX_bl'] = data['DX_bl'].replace(['CN' ,'AD' ,'LMCI' ,'EMCI' ,'SMC'],['0','1','2','3','4']).astype(int)
df['DX'] = df['DX'].replace(['NL' ,'Dementia', 'MCI', 'NL to MCI', 'MCI to Dementia', 'MCI to NL','Dementia to MCI','nan' ,'NL to Dementia'],['1','3','4','5','6','7','8','9','10'])
df['DX']=df['DX'].fillna(0)
print(data['DX_bl'].head())
print(data['DX'].unique())

# drop the columns in data they ate objet type
data = data.select_dtypes(exclude=['object'])
print(data.shape)


In [ ]:
data.info()

In [ ]:
# Algorithme des k plus proches voisins
model = KNeighborsClassifier()
y = data['DXCHANGE']
X = data.drop('DXCHANGE', axis=1)
y = np.nan_to_num(y)
X= np.nan_to_num(X)


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

model.fit(X_train,y_train)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))

In [ ]:
param_grid = {'n_neighbors': np.arange(1, 25),'metric': ['euclidean', 'manhattan']}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)
print(grid.best_estimator_)
model = grid.best_estimator_
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))

In [ ]:
confusion_matrix(y_test, model.predict(X_test))


In [ ]:
N, train_score,val_score = learning_curve(model,X_train,y_train,train_sizes = np.linspace(0.2,1.0,5),cv=5)

In [ ]:
print(N)
plt.plot(N,train_score.mean(axis=1), label='train')
plt.plot(N,val_score.mean(axis=1), label='validation')
plt.xlabel('train_sizes')
plt.legend()


In [ ]:
cv =  GroupKFold(5).get_n_splits(X,y,groups=X[:,5])
cross_val_score(model,X,y,cv=cv)

In [ ]:
def sick(model,PTGENDER=1,AGE=10,PTEDUCAT=4,APOE4=0):
    x= np.array([PTGENDER,AGE,PTEDUCAT,APOE4]).reshape(1,4)
    print(model.predict(x))
    print(model.predict_proba(x))